In [182]:
import pandas as pd
import math
import numpy as np
from statsmodels.tsa.arima_model import ARMA

from sklearn import metrics
from sklearn.metrics import mean_squared_error,r2_score

import warnings
warnings.filterwarnings('ignore')

In [183]:
file=pd.ExcelFile('Test_dataset.xlsx')
df=file.parse('Put-Call_TS',header=[0,1])
df.head(10)

Stock Index      Put-Call Ratio                      \
  Unnamed: 0_level_1 2020-08-10 00:00:00 2020-08-11 00:00:00   
0             AC3235                0.80               0.660   
1             AC3236                0.86               0.732   
2             AC3237                0.93               0.816   
3             AC3238                1.09               1.008   
4             AC3239                 NaN               1.104   
5             AC3240                 NaN               1.152   
6             AC3241                1.23               1.176   
7             AC3242                1.00               0.900   
8             AC3243                1.00               0.900   
9             AC3244                1.07               0.984   

                                                               \
  2020-08-12 00:00:00 2020-08-13 00:00:00 2020-08-14 00:00:00   
0             0.77240            0.983224            0.901211   
1             0.85448            1.086645            1.017042   
2             0.95024            1.207302                 NaN   
3             1.16912            1.483091            1.461062   
4             1.27856            1.620986            1.615504   
5             1.33328            1.689933            1.692725   
6             1.36064            1.724406            1.731335   
7             1.04600            1.327960            1.287315   
8             1.04600            1.327960            1.287315   
9                 NaN            1.448618                 NaN   

                       
  2020-08-15 00:00:00  
0            0.898368  
1            1.029258  
2            1.181962  
3            1.531000  
4            1.705519  
5            1.792779  
6            1.836409  
7            1.334666  
8            1.334666  
9            1.487370

In [184]:
#df.drop('index')
#df.columns.set_index(['Stock Index','Put-Call Ratio'],level=0,inplace=True)
df.columns=['Stock_Index','10-Aug','11-Aug','12-Aug','13-Aug','14-Aug','15-Aug']

In [185]:
df.describe()

,10-Aug,11-Aug,12-Aug,13-Aug,14-Aug,15-Aug
count,2902.000000,3329.000000,3274.000000,3317.000000,3226.000000,3092.000000
mean,0.938828,0.825080,0.960627,1.220309,1.165254,1.201216
std,0.201421,0.242701,0.276744,0.348401,0.390495,0.441742
min,0.600000,0.420000,0.498800,0.638488,0.515107,0.462070
25%,0.762500,0.612000,0.717680,0.914277,0.823990,0.811109
50%,0.930000,0.816000,0.950240,1.207302,1.152179,1.181962
75%,1.110000,1.032000,1.196480,1.517565,1.499673,1.574630
max,1.290000,1.248000,1.442720,1.827827,1.847166,1.967298


In [186]:
df.head(10)

,Stock_Index,10-Aug,11-Aug,12-Aug,13-Aug,14-Aug,15-Aug
0,AC3235,0.80,0.660,0.77240,0.983224,0.901211,0.898368
1,AC3236,0.86,0.732,0.85448,1.086645,1.017042,1.029258
2,AC3237,0.93,0.816,0.95024,1.207302,NaN,1.181962
3,AC3238,1.09,1.008,1.16912,1.483091,1.461062,1.531000
4,AC3239,NaN,1.104,1.27856,1.620986,1.615504,1.705519
5,AC3240,NaN,1.152,1.33328,1.689933,1.692725,1.792779
6,AC3241,1.23,1.176,1.36064,1.724406,1.731335,1.836409
7,AC3242,1.00,0.900,1.04600,1.327960,1.287315,1.334666
8,AC3243,1.00,0.900,1.04600,1.327960,1.287315,1.334666
9,AC3244,1.07,0.984,NaN,1.448618,NaN,1.487370


In [187]:
new=df[['10-Aug','11-Aug','12-Aug','13-Aug','14-Aug','15-Aug']]
#new['mean'] =new.mean(axis = 1, skipna = True)  
#new.head(10)

In [188]:
m = new.mean(axis=1)
for i, col in enumerate(new):
    if col=='Stock Index':
        continue
    new.iloc[:, i] = new.iloc[:, i].fillna(m)
new

,10-Aug,11-Aug,12-Aug,13-Aug,14-Aug,15-Aug
0,0.800000,0.660,0.77240,0.983224,0.901211,0.898368
1,0.860000,0.732,0.85448,1.086645,1.017042,1.029258
2,0.930000,0.816,0.95024,1.207302,1.017101,1.181962
3,1.090000,1.008,1.16912,1.483091,1.461062,1.531000
4,1.464914,1.104,1.27856,1.620986,1.615504,1.705519
...,...,...,...,...,...,...
3326,0.860000,0.732,0.85448,1.086645,1.017042,1.029258
3327,0.640000,0.468,0.55352,0.707435,0.592327,0.549330
3328,0.670000,0.504,0.59456,0.759146,0.650243,0.614775
3329,0.790000,0.648,0.75872,0.965987,0.881906,0.876553


In [190]:
n=len(df)
temp=[]
for j in range(n):
    #print(new.iloc[j,:])
    arr=np.array(new.iloc[j,:-1])
    model = ARMA(arr,order=(0,1,))
    model_fit = model.fit()
    # make prediction
    yhat = model_fit.predict()
    #print(yhat)
    temp.append(yhat.mean())
    
y_pred=pd.DataFrame(temp)

print("mean_sq_err:", mean_squared_error(new['15-Aug'],y_pred))
print('Test Variance score: %.2f' % r2_score(new['15-Aug'], y_pred))

mean_sq_err: 0.04820668012450295
Test Variance score: 0.75


In [191]:
temp=[]
for j in range(n):
    #print(new.iloc[j,:])
    arr=np.array(new.iloc[j,:])
    model = ARMA(arr,order=(0,1,))
    model_fit = model.fit()
    # make prediction
    yhat = model_fit.predict()
    #print(yhat)
    temp.append(yhat.mean())
new['16-Aug']=pd.DataFrame(temp)
new.head()

,10-Aug,11-Aug,12-Aug,13-Aug,14-Aug,15-Aug,16-Aug
0,0.800000,0.660,0.77240,0.983224,0.901211,0.898368,0.839386
1,0.860000,0.732,0.85448,1.086645,1.017042,1.029258,0.932492
2,0.930000,0.816,0.95024,1.207302,1.017101,1.181962,1.015741
3,1.090000,1.008,1.16912,1.483091,1.461062,1.531000,1.288921
4,1.464914,1.104,1.27856,1.620986,1.615504,1.705519,1.478664


In [192]:
df['16-Aug']=new['16-Aug']
df.head()

,Stock_Index,10-Aug,11-Aug,12-Aug,13-Aug,14-Aug,15-Aug,16-Aug
0,AC3235,0.80,0.660,0.77240,0.983224,0.901211,0.898368,0.839386
1,AC3236,0.86,0.732,0.85448,1.086645,1.017042,1.029258,0.932492
2,AC3237,0.93,0.816,0.95024,1.207302,NaN,1.181962,1.015741
3,AC3238,1.09,1.008,1.16912,1.483091,1.461062,1.531000,1.288921
4,AC3239,NaN,1.104,1.27856,1.620986,1.615504,1.705519,1.478664


In [198]:
data = pd.read_excel('Test_dataset.xlsx',name='Test_Aug10')
data['16-Aug']=df['16-Aug']
data=data.drop('Put-Call Ratio',axis=1)
data.rename(columns={'16-Aug':'Put-Call Ratio'},inplace=True)
data.head(10)

,Stock Index,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,P/B Ratio,Put-Call Ratio
0,AC3235,NSE,Materials,53.30,11270.15,44.59,185.09,12067855.0,2.78,0.03,0.0330,74.9,4.66,0.839386
1,AC3236,JSE,Energy,749.34,55722.00,74.20,34.01,6172474.0,6.78,0.11,0.0464,17.7,6.11,0.932492
2,AC3237,S&P 500,Information Tech,567.75,3351.28,88.41,177.40,17472488.0,4.36,0.23,0.0401,1.0,4.99,1.015741
3,AC3238,NSE,Healthcare,646.78,11270.15,79.36,105.44,15553159.0,2.77,0.78,0.0187,74.9,1.26,1.288921
4,AC3239,NYSE,Materials,380.33,12765.84,19.90,139.40,12525784.0,5.93,0.03,0.0261,1.0,6.18,1.478664
5,AC3240,NSE,Healthcare,301.09,11270.15,25.27,149.36,9878930.0,8.17,0.78,0.0233,74.9,1.12,1.546747
6,AC3241,BSE,Materials,227.28,38182.08,38.50,193.01,4934899.0,4.91,0.03,0.0265,74.9,4.60,1.506053
7,AC3242,S&P 500,Real Estate,NaN,NaN,73.40,121.63,4070029.0,6.11,-0.43,0.0504,1.0,4.72,1.149357
8,AC3243,NSE,Energy,NaN,NaN,54.33,169.47,13609149.0,4.93,0.11,0.0609,74.9,3.96,1.149357
9,AC3244,NSE,Healthcare,727.98,11270.15,99.59,139.23,12499327.0,4.65,0.78,0.0206,74.9,2.58,1.244160


In [199]:
data.to_excel("Test_dataset1.xlsx",sheet_name="Test_Aug16")